100学生100题，pymc求难度及能力

In [2]:
from __future__ import division
import os
import sys
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
%precision 4
plt.style.use('ggplot')
np.random.seed(1234)
import pymc
import scipy.stats as stats

In [4]:
c=0.25
D=1.7
n_items=100
n_stu=100
real_a = np.random.lognormal(0,0.2,(n_items,))
real_b = np.random.normal(0,1,(1,n_items))
real_theta = np.random.normal(0,1,(n_stu,1))
uv=np.random.random((n_items,n_stu))<c + (1.0-c) / (1.0 + np.exp(-D*real_a*(real_theta-real_b)))
mask=np.random.random(uv.shape)<0.7
def absm(a,b):
    return np.abs(a-b).mean()

def uirt(uv,mask):
    global c
    global D
    n_stu,n_items=uv.shape
    
    # 难度参数
    b=pymc.Normal('b',mu=0,tau=1,value=np.zeros((1,n_items,)))
    # 能力参数
    theta=pymc.Normal('theta',mu=0,tau=1,value=np.zeros((n_stu,1)))
    # 区分度参数
    a=pymc.Lognormal('a',mu=0,tau=25,value=np.ones((n_items,)))
    
    

    @pymc.deterministic
    def sigmoid(theta=theta,b=b,a=a): 
        global c
        global D
        return c + (1.0-c) / (1.0 + np.exp(-D*a*(theta-b)))
    # 答题矩阵
    u=pymc.Bernoulli('u', p=sigmoid, value=uv, observed=True)
    
    params=[a,b,theta,sigmoid]
    
    if mask is None:
        params.append(u)
    else:        
        @pymc.stochastic(dtype=np.bool,observed=True)
        def uv_miss(value=uv, mask=mask,p=sigmoid):
            """answer matrix"""

            def logp(value, mask,p):
                lh=-value*np.log(p)-(1-value)*np.log(1-p)
                return (lh*mask).sum()


            def random(mask,p):
                return np.random.random(mask.shape)<p
        params.append(u)        

    m=pymc.MCMC(params)
    m.sample(100000,40000,12)
    
    
    ea=a.stats()['mean']
    eb=b.stats()['mean']
    et=theta.stats()['mean']
    m.stats()
    print absm(ea.reshape(real_a.shape),real_a),absm(eb.reshape(real_b.shape),real_b),absm(et.reshape(real_theta.shape),real_theta)
    return ea,eb,et,
uirt(uv,mask)

C:\Program1\Anaconda2\lib\site-packages\ipykernel\__main__.py:45: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


 [-----------------100%-----------------] 100000 of 100000 complete in 294.4 sec0.175190538441 0.683862395218 0.741809995854


(array([ 1.038 ,  1.0404,  1.0758,  0.9017,  0.9889,  0.9754,  1.1162,
         0.9874,  0.9132,  0.883 ,  0.9643,  1.0591,  1.0673,  1.0502,
         1.0683,  1.1209,  1.0138,  1.1145,  1.0271,  0.9043,  0.9884,
         0.9564,  1.0372,  0.9406,  1.06  ,  0.8154,  0.9338,  1.0327,
         1.0684,  0.9399,  1.0687,  1.0518,  1.024 ,  1.02  ,  0.9434,
         1.0094,  1.0439,  0.8318,  0.9069,  0.9313,  0.8809,  0.8655,
         1.0171,  0.9246,  0.9999,  1.0721,  1.0611,  0.9208,  1.1002,
         1.064 ,  0.9498,  0.7583,  0.9906,  0.9077,  0.9843,  1.0462,
         0.8985,  1.0832,  1.1318,  0.8713,  0.9591,  0.9917,  1.0952,
         1.0558,  1.1231,  1.0515,  1.0616,  1.1499,  1.0934,  0.8789,
         1.0068,  1.0831,  0.8808,  1.1483,  0.8995,  0.9155,  0.9281,
         1.0706,  1.0892,  1.0265,  1.0377,  1.0043,  1.0061,  1.1475,
         1.0179,  1.0556,  0.9705,  1.0508,  1.0905,  0.8883,  1.1087,
         1.0157,  0.9379,  0.9724,  1.0433,  0.9191,  1.0429,  1.1244,
      